## Create all image path

In [20]:
import glob
import json

jpg_files_path = glob.glob('images/Keyframes/**/keyframes/**/*.jpg', recursive=True)

jpg_dict = {i:jpg_path for i, jpg_path in enumerate(jpg_files_path)}

jpg_sorted_dict = dict(sorted(jpg_dict.items(), key=lambda item: item[1]))

reset_jpg_sorted_dict = {i: v for i, (k, v) in enumerate(jpg_sorted_dict.items())}

for i, value in reset_jpg_sorted_dict.items():
    print(i, value)

    if i == 5:
        break


0 images/Keyframes/Keyframes_L01/keyframes/L01_V001/001.jpg
1 images/Keyframes/Keyframes_L01/keyframes/L01_V001/002.jpg
2 images/Keyframes/Keyframes_L01/keyframes/L01_V001/003.jpg
3 images/Keyframes/Keyframes_L01/keyframes/L01_V001/004.jpg
4 images/Keyframes/Keyframes_L01/keyframes/L01_V001/005.jpg
5 images/Keyframes/Keyframes_L01/keyframes/L01_V001/006.jpg


In [22]:
file_name = "image_all_path.json"

# Open the file in write mode and write the JSON data
with open(file_name, 'w') as json_file:
    json.dump(reset_jpg_sorted_dict, json_file, indent=4)

## Create faiss bin for keyframes

In [23]:
import torch
from PIL import Image
from transformers import CLIPProcessor, CLIPModel


if torch.cuda.is_available():
    device = torch.device("cuda")
    print("CUDA is available. Using GPU.")
else:
    device = torch.device("cpu")
    print("CUDA is not available. Using CPU.")


CUDA is not available. Using CPU.


In [24]:
# Load CLIP model and processor
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

In [25]:
# Check if the model is on the GPU
if next(model.parameters()).is_cuda:
    print("The model is running on GPU.")
else:
    print("The model is running on CPU.")


The model is running on CPU.


In [26]:
def embed_images(image_paths):
    # Load and preprocess images
    images = [Image.open(path) for path in image_paths]
    
    # Preprocess the images using the CLIP processor and move to GPU
    inputs = processor(images=images, return_tensors="pt", padding=True)
    
    # Move inputs to the same device as the model (GPU if available)
    inputs = {k: v.to(device) for k, v in inputs.items()}

    # Generate embeddings using the GPU
    with torch.no_grad():
        image_features = model.get_image_features(**inputs)
    
    # Normalize the embeddings
    image_features = image_features / image_features.norm(dim=-1, keepdim=True)
    
    # Convert the embeddings to CPU before converting to NumPy, to avoid GPU memory issues
    return image_features.cpu().numpy()


In [27]:
import json

# Replace 'your_file.json' with the path to your JSON file
json_file_path = 'image_all_path.json'

# Open and load the JSON file
with open(json_file_path, 'r') as file:
    image_paths = json.load(file)

# Print the dictionary to verify
# print(image_paths)


In [ ]:
import faiss
import numpy as np
from tqdm import tqdm

# Initialize FAISS index (assuming we have 512-dim CLIP features)
dimension = 512  # CLIP model output dimension
index = faiss.IndexFlatL2(dimension)

# Batch processing (if you have a large number of images, process them in batches)
batch_size = 16  # Adjust the batch size depending on your memory
image_keys = list(image_paths.keys())  # Get all keys from the dictionary

for i in tqdm(range(0, len(image_keys), batch_size), desc="Embedding Images"):
    # Get the current batch of image paths
    batch_keys = image_keys[i:i + batch_size]
    batch_paths = [image_paths[key] for key in batch_keys]
    
    # Embed the images using the GPU
    embeddings = embed_images(batch_paths)
    
    # Add the batch of embeddings to the FAISS index
    index.add(embeddings)

print(f"Total embeddings indexed: {index.ntotal}")


Embedding Images:   0%|          | 7/6662 [00:05<1:29:36,  1.24it/s]

In [ ]:
# Save the FAISS index to a file
faiss.write_index(index, "faiss_index.bin")


In [ ]:
# Load the FAISS index from a file
index = faiss.read_index("faiss_index.bin")
index